In [ ]:
import openai
import datetime
import concurrent.futures
import os
import time

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
def send_message(message, temperature=0.7):
    try:
        result = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            # model="gpt-4",
            temperature=temperature,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": message}
            ]
        )
        return result.choices[0].message.content
    except Exception as e:
        print(f"Error in send_message: {e}")
        return None

def retry(func, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            return func()
        except Exception as e:
            print(f"Error: {e}, retrying... ({retries + 1}/{max_retries})")
            retries += 1
            time.sleep(1)  # Wait a bit before retrying
    raise Exception(f"Failed after {max_retries} retries.")

def step_by_step_3x(question):
    question_and_prompt = f"""
                    {question} Let's work this out in a step by step way to be sure we have the right answer
                """
    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(send_message, question_and_prompt, 0.9) for _ in range(3)]
            responses = [future.result() for future in concurrent.futures.as_completed(futures)]
    except:
        # If concurrent execution fails, try sequentially
        print("Error with concurrent execution. Switching to sequential execution.")
        responses = [retry(lambda: send_message(question_and_prompt, 0.9)) for _ in range(3)]

    return responses


In [ ]:
def researcher(response_1, response_2, response_3):
    researcher_prompt = f"""
    You are a researcher tasked with investigating the 3 response options provided.  
    List the flaws and faulty logic of each answer option. Let's work this out in a 
    step by step way to be sure we have all the errors.
    """
    researcher_plus_responses = researcher_prompt + "response 1: " + response_1 + "response 2: " + response_2 + "response 3: " + response_3
    result = retry(lambda: send_message(researcher_plus_responses))
    return result
    

In [ ]:
def resolver(researcher_response, response_1, response_2, response_3):
    resolver_prompt = f"""
    You are a resolver tasked with 1) finding which of the 3 answer options the 
    researcher thought was best. 2) improving that answer, and 3) Printing the improved
    answer in full.  Let's work this out in a step by step way to be sure
    we have the right answer.
    """
    resolver_prompt_plus_responses = resolver_prompt + "researcher criticisms: " + researcher_response + "\n orginal responses: \n" + "response 1: " + response_1 + "response 2: " + response_2 + "response 3: " + response_3
    result = retry(lambda: send_message(resolver_prompt_plus_responses))
    return result

In [ ]:
def save_to_file(final_result, question, response_1, response_2, response_3, researcher_result):
    """Save the result and intermediate steps to a file."""
    dir_name = "logs"
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    question_truncated = question.replace(" ", "_")
    question_truncated = question_truncated[:40]
    now = datetime.datetime.now()
    now = now.strftime("%Y-%m-%d_%H-%M-%S")
    with open(f"logs/{question_truncated}_{now}.md", "w") as f:
        f.write(f"""
# Question: {question}
# responses:
## response 1: 
    {response_1}
## response 2:
    {response_2}
## response 3:
    {response_3}
# researcher result:
    {researcher_result}
# final result:
    {final_result}"""
)


In [ ]:
def full_pipeline(question):
    print("asking question 3x. question: ", question)
    response_1, response_2, response_3 = step_by_step_3x(question)
    print("researcher step")
    researcher_result = researcher(response_1, response_2, response_3)
    print("resolver step")
    final_result = resolver(researcher_result, response_1, response_2, response_3)
    save_to_file(final_result, question, response_1, response_2, response_3, researcher_result)
    return final_result

In [ ]:
question = input("Please enter your quesiton for the research and resolver tool: ")
print(f"You entered: {question}")
result = full_pipeline(question)
print(result)